In [10]:
import os
import pandas as pd

In [11]:
demand_train_A = 'data/demand_train_A.csv'
geo_topo = 'data/geo_topo.csv'
inventory_info_A = 'data/inventory_info_A.csv'
product_topo = 'data/product_topo.csv'
weight_A = 'data/weight_A.csv'

demand_train_A = pd.read_csv(demand_train_A)
geo_topo = pd.read_csv(geo_topo)
inventory_info_A = pd.read_csv(inventory_info_A)
product_topo = pd.read_csv(product_topo)
weight_A = pd.read_csv(weight_A)

demand_test_A = 'data/demand_test_A.csv'

demand_test_A = pd.read_csv(demand_test_A)

dfs = [demand_train_A,geo_topo,inventory_info_A,product_topo,weight_A,demand_test_A]
for df in dfs:
    if 'Unnamed: 0' in df.columns:
        df.drop(columns='Unnamed: 0',inplace=True)
    if 'ts' in df.columns:
        df = df.sort_values(by='ts')

In [12]:
test_ts_start = demand_test_A.ts.min()

In [13]:
all_data = pd.concat([demand_train_A,demand_test_A])
all_data = all_data.sort_values(by='ts')
all_data = all_data.reset_index().drop(columns='index')

In [14]:
all_data.head()

,unit,ts,qty,geography_level,geography,product_level,product
0,9b8f48bacb1a63612f3a210ccc6286cc,2018-06-04,11926.8286,geography_level_3,36ab7b000da26b0547bfc3c3fdf143dc,product_level_2,5cc8015f03554313900f069182bdaf9c
1,4d3ca213b639541c5ba4cf8a69b1e1ed,2018-06-04,628.1582,geography_level_3,7ac6047d36cb2b463fc6b483cc32da60,product_level_2,5cc8015f03554313900f069182bdaf9c
2,06531cd4188630ce2497cd9983aacf5e,2018-06-04,6515.8894,geography_level_3,5f82497e6ba74177eacda9f48d2ebb8f,product_level_2,5cc8015f03554313900f069182bdaf9c
3,326cb18b045e5baefa90bbc2e8d52a32,2018-06-04,12311.5442,geography_level_3,60de5fa40aaaf2850711bc5269f91476,product_level_2,5cc8015f03554313900f069182bdaf9c
4,04b155599672da2f35abd187df4b7d3f,2018-06-04,11534.1116,geography_level_3,36ab7b000da26b0547bfc3c3fdf143dc,product_level_2,ae7d2d22786beebc5981b93c226e2aba


### 预测未来需求量

In [15]:
## 真实值串14天
submission = demand_test_A
submission['yesterday_qty'] = submission.groupby('unit')['qty'].shift(1).fillna(method='ffill').reset_index().sort_index().set_index('index')

submission['diff_1'] = submission['qty'] - submission['yesterday_qty']
submission['qty'] = submission['diff_1']
# submission.loc[submission['qty']<0,'qty']=0

submission['shift_14']=submission.groupby('unit')['qty'].shift(-14).fillna(0).reset_index().sort_index().set_index('index')
submission = submission[['unit','ts','shift_14']].rename(columns={'shift_14':'qty'})


# 按照7天聚合
submission['dt'] = pd.to_datetime(submission['ts'])
submission['weekofyear'] = submission['dt'].dt.weekofyear
submission['year'] = submission['dt'].dt.year
submission_week = submission.copy()
submission_week = submission_week.groupby(['weekofyear','year','unit'],as_index=False).sum()
submission_week['sum_qty'] = submission_week['qty']
submission = pd.merge(submission_week,submission,on = ['weekofyear','year','unit'])
submission['dayofweek'] = submission['dt'].dt.dayofweek
submission = submission[submission['dayofweek']==0]
submission = submission[['unit','ts','sum_qty']].rename(columns={'sum_qty':'qty'})


<ipython-input-15-bb7a9c5c9b63>:15: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  submission['weekofyear'] = submission['dt'].dt.weekofyear


### 根据未来需求消耗掉初始库存

In [16]:
init_inventory = inventory_info_A.set_index(['unit'])['qty'].to_dict()
def consume_init_inventory(arr,init_val):
    remain = init_val
    i = 0
    while remain>0 and i<len(arr):
        arr[i] = max(0,arr[i]-remain)
        remain -= arr[i]
        i+=1
    return arr

r = []
for i,group in submission.groupby('unit'):

    unit = group['unit'].values[0]
    init_val = init_inventory[unit]
    
    group = group.sort_values(by='ts')
    qty_list = group['qty'].values
    qty_list = consume_init_inventory(qty_list,init_val)
    group['qty'] = qty_list
    r.append(group)

    
submission = pd.concat(r)    
    

In [17]:
submission.to_csv('submission.csv',index=False)

In [9]:
submission

,unit,ts,qty
3792,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-08,0.000000
8216,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-15,0.000000
12640,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-22,0.000000
17064,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-29,0.000000
21488,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-04-05,0.000000
...,...,...,...
48025,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-10,15.333333
52449,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-17,1002.333333
56873,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-24,-0.333333
61297,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-31,0.000000


In [18]:
submission

,unit,ts,qty
3792,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-08,0.000000
8216,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-15,0.000000
12640,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-22,0.000000
17064,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-03-29,0.000000
21488,0025accbb2e3dfbfe6f5b3a4562bdee0,2021-04-05,0.000000
...,...,...,...
48025,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-10,15.333333
52449,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-17,1002.333333
56873,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-24,-0.333333
61297,ffddc0dbb7fa28b00a5b6ddc8e7e317c,2021-05-31,0.000000
